<a href="https://colab.research.google.com/github/nicolasquant/inteligencia_artificial/blob/main/main_previsao_temporal_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#lendo a base de dados
df_acoes = pd.read_csv("Data_trat.csv")

n_dias = 7   # qtd de dias p analisar
corte = int(len(df_acoes.index)*0.7) # ponto de corte entre treino e teste
quantia = 10000 # quantia para investimento
dif = 0
dias_test = 21  # dias de trade
valor_acoes = 0 # variavel que marca o valor total em açoes na carteira
# obs. para acharmos o nome do rotulo (label) de uma coluna, usamos df_acoes.columns[1] e o retorno sera, nesse caso: ALOS3.SA

# retirando as colunas onde o valor da ação é igual a 0 (aqui estamos observando o dia 1794 em diante)
j = 0
for i in range(len(df_acoes.columns)):
  if df_acoes.iloc[(corte-n_dias), i-j] == 0.0: # queremos remover todas as colunas onde o valor da acao seja=0 (estamos pegando o dia relacionado a 70% do banco de dados)
      df_acoes.drop(df_acoes.columns[i-j], axis=1, inplace=True) # df_acoes é o mesmo de usar inplace=True - inplace=True significa que a remoção será feita diretamente no DataFrame original, sem necessidade de criar uma cópia. Se fosse inplace=False, uma nova cópia do DataFrame com a linha removida seria retornada
      j+=1

# n_ativos = len(df_acoes.columns) # aqui possuimos 100 ativos
n_ativos = 25
# declarando os vetores
predicao = np.zeros((n_ativos), "float32")
carteira = np.zeros((n_ativos), "int32")
custo = np.zeros((n_ativos), "float32")

# para cada dia analisado é necessário treinar a rede com os ultimos 7 dias
for d in range(dias_test):
    #atualizando o corte a cada dia
    corte = corte + d
    print(f"dia {d} linha {corte}")

    #preenchendo variaveis
    train_df = df_acoes.loc[:corte,:]
    test_df = df_acoes.loc[corte:,:]
    train_day = np.zeros((corte), "int32")
    test_day = np.zeros((len(df_acoes.index) - corte), "int32")
    for i in range(len(df_acoes.index)):
        if i < corte:
            train_day[i] = i
        else:
            indice = i - corte
            test_day[indice] = i

    # treinando
    for i in range(n_ativos):
        train_norm = tf.keras.layers.Normalization(input_shape=[1,], axis=None) # Primeira camada: define a forma da entrada - aqui temos inicialemnte 7 pesos, e não 1 peso para 7 dados
        train_norm.adapt(np.asarray(df_acoes.index).astype("int32")) # estamos nomalizando os dias
        lin_model = tf.keras.Sequential([ # kera.sequential ordena a relação entre as camadas que estamos criando, de forma sequencial.
            train_norm, # essa é nossa primeira camada e possui 7 pesos referentes aos 7 dados de entradas dados pela nossa matriz com os ultimos dias da matriz train_day
            tf.keras.layers.Dense(units=1)
            ])
        lin_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.05),
        loss='mean_absolute_error',
        metrics=['MeanAbsolutePercentageError'])
        lin_model.fit(np.asarray(train_day[-(n_dias+2):]).astype('float32'), # estamos acessando os ultimos 9 elementos da array com -(n_dias +2)
            np.asarray(train_df.iloc[-(n_dias+2):,i]).astype('float32'),
            epochs=300, # 2100 atualizações de pesos
            verbose=0,
            validation_split = 0.2, # soma dois no treino para sobrar dados para a validação
            callbacks=[tf.keras.callbacks.ModelCheckpoint(filepath="pesosRec" + str(n_dias) + "/training_ML_" + str(i) + "/cp.ckpt",
                                                    save_weights_only=True,
                                                    verbose=0)])
    # avaliando
    print("test_day ", type(test_day[0]))
    for i in range(n_ativos):
        # print("ativo", i)
        lin_model.load_weights("pesosRec" + str(n_dias) + "/training_ML_" + str(i) + "/cp.ckpt")
        predicao[i] = lin_model.predict(np.asarray([test_day[0],]))
        # se a reta estiver ascendente...
        if lin_model.predict(np.asarray([test_day[1],])) > predicao[i]:
            # diferença do valor dado pela reta com o valor atual
            dif = predicao[i] - test_df.iloc[0,i]
            # se a reta estiver acima do valor da ação, futuramente ela deve subir
            if dif > 0:
                #compra
                for c in range(10):
                    if quantia >= test_df.iloc[0,i]:
                        carteira[i] +=1
                        quantia -= test_df.iloc[0,i]
                        custo[i] += test_df.iloc[0,i]
                    else:
                        break
            # caso contrário ela deve descer
            else:
                # venda
                if carteira[i] > 0:
                    quantia += (test_df.iloc[0, i] * carteira[i])
                    custo[i] = 0
                    carteira[i] = 0

    #calculo dos valores atuais da carteira a cada dia
    for i in range(len(carteira)):
        valor_acoes += (test_df.iloc[0, i] * carteira[i])
    print(f"quantia: {quantia}")
    print(f"valor total: {quantia + valor_acoes}")
    valor_acoes = 0
    print(carteira)

dia 0 linha 1794
test_day  <class 'numpy.int32'>
1/1 [==============================] - 0s 19ms/step


<ipython-input-4-b86bbdfc8311>:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicao[i] = lin_model.predict(np.asarray([test_day[0],]))


1/1 [==============================] - 0s 18ms/step
quantia: 8037.01789855957
valor total: 10000.0
[10  0  0 10  0  0 10  0  0 10 10  0 10 10 10  0 10 10  0 10 10 10 10  0
 10]
dia 1 linha 1795
test_day  <class 'numpy.int32'>
1/1 [==============================] - 0s 20ms/step
quantia: 6686.034932136536
valor total: 10004.23894405365
[20  0  0  0  0  0 20  0  0 20 20  0 20  0 20  0 20 20  0 20 20 20 20  0
 20]
dia 2 linha 1797
test_day  <class 'numpy.int32'>
1/1 [==============================] - 0s 19ms/step
quantia: 8097.948100566864
valor total: 10104.338598251343
[ 0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0 30 30  0  0 30 30  0  0
 30]
dia 3 linha 1800
test_day  <class 'numpy.int32'>
1/1 [==============================] - 0s 19ms/step
quantia: 9338.287830352783
valor total: 10154.302501678467
[ 0  0  0  0  0  0  0 10 10 10  0  0  0  0 10  0  0  0  0  0  0  0 10  0
  0]
dia 4 linha 1804
test_day  <class 'numpy.int32'>
1/1 [==============================] - 0s 27ms/step
quantia: 